In [ ]:
import numpy as np

from zdm import loading
from zdm import parameters
from astropy.cosmology import Planck18
from zdm import misc_functions as mf

from zdm import iteration as iteration

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['figure.figsize'] = [8,6]

In [ ]:
file = 'MC_CRAFT_ICS_1300_2'
# file = 'DSA'

state = parameters.State()
state.set_astropy_cosmo(Planck18)

param_dict={'sfr_n': 0.8808527057055584, 'alpha': 0.7895161131856694, 'lmean': 2.1198711983468064, 'lsigma': 0.44944780033763343, 
            'lEmax': 41.18671139482926, 'lEmin': 39.81049090314043, 'gamma': -1.1558450520609953, 'H0': 54.6887137195215, 'sigmaDMG': 0.0, 'sigmaHalo': 0.0}
state.update_params(param_dict)

ss,gs = loading.surveys_and_grids(survey_names=[file], init_state=state, rand_DMG=False)

s = ss[0]
g = gs[0]

iteration.get_log_likelihood(g,s)

In [ ]:
if s.zlist is None:
    Zs = None
    DMs = None
else:
    Zs = s.Zs[s.zlist]
    DMs = s.DMEGs[s.zlist]

if s.nozlist is not None:
    DMlines = s.DMEGs[s.nozlist]
else:
    DMlines = None

mf.plot_grid_2(
    zDMgrid=g.rates,
    zvals=g.zvals,
    dmvals=g.dmvals,
    zmax=2.5,
    DMmax=3500,
    norm=0,
    log=True,
    project=False,
    Aconts=[0.01,0.1,0.5],
    showplot=True,
    save=False,
    name=file + ".pdf",
    DMlines=DMlines,
    DMlims=[s.max_dm],
    FRBZ=Zs,
    FRBDM=DMs,
    data_clr='r',
    data_clr2='tab:blue'
)

In [ ]:
xedges = np.append(g.zvals[::10], g.zvals[-1])
yedges = np.append(g.dmvals[::10], g.dmvals[-1])
heatmap, xedges, yedges = np.histogram2d(Zs, DMs, bins=(xedges, yedges))
dz = g.zvals[1] - g.zvals[0]
ddm = g.dmvals[1] - g.dmvals[0]

heatmap = heatmap
rates = np.zeros([int(len(g.zvals)/10), int(len(g.dmvals)/10)])

for i in range(g.rates.shape[0]):
    for j in range(g.rates.shape[1]):
        rates[i//10,j//10] += g.rates[i,j]

rates = rates / np.sum(rates) / dz / ddm
print(np.sum(np.abs(heatmap - rates)))

mf.plot_grid_2(
    zDMgrid= np.abs(heatmap),
    zvals=g.zvals[5::10],
    dmvals=g.dmvals[5::10],
    zmax=2.5,
    DMmax=3500,
    norm=0,
    log=True,
    project=False,
    showplot=True,
    save=False,
    name=file + ".pdf",
    DMlims=[s.max_dm],
    data_clr='r',
    data_clr2='tab:blue'
)

In [ ]:
pdm = np.sum(g.rates, axis=0)
pz = np.sum(g.rates, axis=1)

fig, axs = plt.subplots(2,1)
dm_hist = axs[0].hist(s.DMEGs, bins=100)
axs[0].plot(g.dmvals, pdm / np.max(pdm) * np.max(dm_hist[0]))
axs[0].set_xlim(left=0, right=2000)

z_hist = axs[1].hist(s.Zs, bins=100)
axs[1].plot(g.zvals, pz / np.max(pz) * np.max(z_hist[0]))
axs[1].set_xlim(left=0, right=2)

In [ ]:
# s.Gbs = s.Gbs / np.pi * 180
x = np.cos(s.Gls/180*np.pi) * np.cos(s.Gbs/180*np.pi)
y = np.sin(s.Gls/180*np.pi) * np.cos(s.Gbs/180*np.pi)
z = np.sin(s.Gbs/180*np.pi)

fig = plt.figure()

ax  = fig.add_subplot(1, 1, 1, projection='3d')

ax.plot(x, y, z, '.k')

ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_zlim(-1.1, 1.1)

ax.view_init(elev=30, azim=15)

In [ ]:
hist = plt.hist(s.DMGs, bins=100)

# high = np.sin(s.Gbs[s.DMGs > 100]/180*np.pi)
# plt.hist(high, bins=20)

In [ ]:
plt.hist(np.sin(s.Gbs/180*np.pi), bins=20)

In [ ]:
plt.hist(s.Gls)

In [10]:
# uDMGs = 0.5
# # DMhalo = 100.0

# ax = plt.subplot()
# plt.title(s.name)
# ax.set_xlabel('DM')
# ax.set_ylabel('Weight')

# # s.DMhalo = DMhalo
# # s.init_DMEG(DMhalo)

# dmvals=g.dmvals
# DMobs=s.DMEGs

#     # calc_DMG_weights(DMEGs, DMhalos, DM_ISMs, dmvals, sigma_ISM=0.5, sigma_halo=15.0, percent_ISM=True)
# dm_weights, iweights = iteration.calc_DMG_weights(DMobs, s.DMhalos, s.DMGs, dmvals, uDMGs)
# print(s.DMEGs)

# pdm = np.sum(g.rates, axis=0)
# pdm = pdm / np.max(pdm) * np.max(dm_weights[0])

# for i in range(len(DMobs)):
#     ax.plot(dmvals[iweights[i]], dm_weights[i], '.-', label=s.frbs["TNS"][i] + " " + str(s.DMGs[i]))

# ax.plot(dmvals, pdm) # Upper limit is not correct because grid has not been updated so efficiencies have not been recalc'd
# ax.set_xlim(right=3000)
# # ax.legend()

# iteration.get_log_likelihood(g,s)
